## 2017 Data Import and Cleaning

#### Read In Data

In [1]:
import pandas as pd

In [2]:
# check the file names in the data folder
!ls ../data

act_2017.csv       combined_2017.csv  percent_change.png sat_2018.csv
act_2018.csv       final.csv          sat_2017.csv


In [45]:
# read in the data
act_df = pd.read_csv('../data/act_2017.csv')
sat_df = pd.read_csv('../data/sat_2017.csv')

#### Display Data

In [4]:
# check out the first few rows of the ACT dataframe
act_df.head()

,State,Participation,English,Math,Reading,Science,Composite
0,National,60%,20.3,20.7,21.4,21.0,21.0
1,Alabama,100%,18.9,18.4,19.7,19.4,19.2
2,Alaska,65%,18.7,19.8,20.4,19.9,19.8
3,Arizona,62%,18.6,19.8,20.1,19.8,19.7
4,Arkansas,100%,18.9,19.0,19.7,19.5,19.4


In [5]:
# check out the first few rows of the SAT dataframe
sat_df.head()

,State,Participation,Evidence-Based Reading and Writing,Math,Total
0,Alabama,5%,593,572,1165
1,Alaska,38%,547,533,1080
2,Arizona,30%,563,553,1116
3,Arkansas,3%,614,594,1208
4,California,53%,531,524,1055


#### Describe Data

In [6]:
act_df.shape

(52, 7)

The ACT dataframe has 52 rows and 7 columns.

In [7]:
act_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 52 entries, 0 to 51
Data columns (total 7 columns):
State            52 non-null object
Participation    52 non-null object
English          52 non-null float64
Math             52 non-null float64
Reading          52 non-null float64
Science          52 non-null float64
Composite        52 non-null object
dtypes: float64(4), object(3)
memory usage: 2.9+ KB


The ACT dataframe includes participation rates and English, Math, Reading, Science, and Composite scores for each of the 50 states and Washington DC. It also includes aggregate National data.

In [8]:
sat_df.shape

(51, 5)

The SAT dataframe has 51 rows and 5 columns.

In [9]:
sat_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 5 columns):
State                                 51 non-null object
Participation                         51 non-null object
Evidence-Based Reading and Writing    51 non-null int64
Math                                  51 non-null int64
Total                                 51 non-null int64
dtypes: int64(3), object(2)
memory usage: 2.1+ KB


The SAT dataset includes participation rates and Evidence-Based Reading and Writing, Math, and Total scores for each of the 50 states and Washington DC. It does not include aggregate National data.

#### Begin Cleaning

In [10]:
sat_df.describe()

,Evidence-Based Reading and Writing,Math,Total
count,51.000000,51.000000,51.000000
mean,569.117647,547.627451,1126.098039
std,45.666901,84.909119,92.494812
min,482.000000,52.000000,950.000000
25%,533.500000,522.000000,1055.500000
50%,559.000000,548.000000,1107.000000
75%,613.000000,599.000000,1212.000000
max,644.000000,651.000000,1295.000000


Minimum SAT score for math, 52, appears to be too low, since the minimum score on the test is 200.

In [11]:
# look for math scores scores below 200
sat_df[sat_df.Math < 200]  

,State,Participation,Evidence-Based Reading and Writing,Math,Total
20,Maryland,69%,536,52,1060


In [12]:
# set the score for Maryland to the correct score
# which can be looked up at https://blog.collegevine.com/here-are-the-average-sat-scores-by-state/
sat_df.iloc[20, [3]] = 524

In [13]:
# check that the change was made correctly
sat_df.loc[[20], ['Math']]

,Math
20,524


Turns out the correct value could also have been imputed by looking at the difference between the Total score and the Writing score, but, since the data is readily available, and it was only one value, it made sense to go to the source.

In [14]:
act_df.describe()

,English,Math,Reading,Science
count,52.000000,52.000000,52.000000,52.000000
mean,20.919231,21.173077,22.001923,21.040385
std,2.332132,1.963602,2.048672,3.151113
min,16.300000,18.000000,18.100000,2.300000
25%,19.000000,19.400000,20.475000,19.900000
50%,20.550000,20.900000,21.700000,21.150000
75%,23.300000,23.100000,24.125000,22.525000
max,25.500000,25.300000,26.000000,24.900000


Minimum ACT score for Science, 2.3, appears very low relative to other average scores.

In [15]:
# look at the lowest science scores
act_df[act_df['Science'] < 19]

,State,Participation,English,Math,Reading,Science,Composite
21,Maryland,28%,23.3,23.1,24.2,2.3,23.6
25,Mississippi,100%,18.2,18.1,18.8,18.8,18.6
29,Nevada,100%,16.3,18.0,18.1,18.2,17.8
41,South Carolina,100%,17.5,18.6,19.1,18.9,18.7


In [16]:
# reset the incorrect score to the correct score,
# which can be found at https://blog.prepscholar.com/act-scores-by-state-averages-highs-and-lows
act_df.loc[[21], ['Science']] = 23.2

In [17]:
# check to see that the score was correctly changed
act_df.loc[[21], ['Science']] 

,Science
21,23.2


#### Check Datatypes

In [18]:
# check SAT datatypes
sat_df.dtypes

State                                 object
Participation                         object
Evidence-Based Reading and Writing     int64
Math                                   int64
Total                                  int64
dtype: object

SAT Participation is object type, but should be numeric.

In [19]:
# check ACT datatypes
act_df.dtypes

State             object
Participation     object
English          float64
Math             float64
Reading          float64
Science          float64
Composite         object
dtype: object

ACT Participation and Composite score are both object type, but should be numeric.

In [20]:
# pd.to_numeric(sat_df.Participation)

# cell returned "ValueError: Unable to parse string "5%" at position 0"

In [21]:
# define a function to remove % symbols and convert to integers
def participation_to_numeric(dataframe, column = 'Participation'):
    dataframe[column] = dataframe[column].map(lambda x: int(x.replace('%', '')))

In [22]:
# pass the dataframes to the function
participation_to_numeric(sat_df)
participation_to_numeric(act_df)

In [23]:
# check that the participation columns were correctly changed to integer type
print(sat_df.dtypes)
print('')
print(act_df.dtypes)

State                                 object
Participation                          int64
Evidence-Based Reading and Writing     int64
Math                                   int64
Total                                  int64
dtype: object

State             object
Participation      int64
English          float64
Math             float64
Reading          float64
Science          float64
Composite         object
dtype: object


In [24]:
# act_df['Composite'] = pd.to_numeric(act_df['Composite'])
# cell returned "ValueError: Unable to parse string '20.2x' at position 51"

In [25]:
# reassign the value so it has only numeric characters
act_df.loc[51, 'Composite'] = '20.2'

In [26]:
# check that the value was correctly changed
act_df.Composite[51]

'20.2'

In [27]:
# run the .to_numeric method on the column again
act_df['Composite'] = pd.to_numeric(act_df['Composite'])

In [28]:
# check that the datatype was changed correctly
act_df.dtypes

State             object
Participation      int64
English          float64
Math             float64
Reading          float64
Science          float64
Composite        float64
dtype: object

#### Rename ACT Columns

In [29]:
# display ACT column names
act_df.columns

Index(['State', 'Participation', 'English', 'Math', 'Reading', 'Science',
       'Composite'],
      dtype='object')

In [30]:
# prepend '2017_act_' to columns other than State, and convert to lowercase
act_df.columns = act_df.columns.map(lambda x: x.lower() if x == 'State' else '2017_act_' + x.lower())
act_df.columns

Index(['state', '2017_act_participation', '2017_act_english', '2017_act_math',
       '2017_act_reading', '2017_act_science', '2017_act_composite'],
      dtype='object')

#### Rename SAT Columns

In [31]:
# display SAT column names
sat_df.columns

Index(['State', 'Participation', 'Evidence-Based Reading and Writing', 'Math',
       'Total'],
      dtype='object')

In [32]:
# prepend '2017_sat_' to columns other than State, and convert to lowercase
sat_df.columns = sat_df.columns.map(lambda x: x.lower() if x == 'State' else '2017_sat_' + x.lower())
sat_df.columns

Index(['state', '2017_sat_participation',
       '2017_sat_evidence-based reading and writing', '2017_sat_math',
       '2017_sat_total'],
      dtype='object')

In [33]:
# replace spaces in '2017_sat_evidence-based reading and writing' with underscores
sat_df.rename(columns={'2017_sat_evidence-based reading and writing':'2017_sat_evidence-based_reading_and_writing' }, inplace=True)
sat_df.columns

Index(['state', '2017_sat_participation',
       '2017_sat_evidence-based_reading_and_writing', '2017_sat_math',
       '2017_sat_total'],
      dtype='object')

#### Drop unnecessary rows

In [34]:
# 
act_df.loc[[0], :]

,state,2017_act_participation,2017_act_english,2017_act_math,2017_act_reading,2017_act_science,2017_act_composite
0,National,60,20.3,20.7,21.4,21.0,21.0


In [35]:
act_df.drop(0, inplace=True)

In [36]:
act_df.head()

,state,2017_act_participation,2017_act_english,2017_act_math,2017_act_reading,2017_act_science,2017_act_composite
1,Alabama,100,18.9,18.4,19.7,19.4,19.2
2,Alaska,65,18.7,19.8,20.4,19.9,19.8
3,Arizona,62,18.6,19.8,20.1,19.8,19.7
4,Arkansas,100,18.9,19.0,19.7,19.5,19.4
5,California,31,22.5,22.7,23.1,22.2,22.8


#### Merge Dataframes and Save

In [43]:
# merge the datasets and assign to a new variable name
combined_2017 = pd.merge(left = act_df, 
                         right = sat_df, 
                         how = 'left', 
                         on='state')

# and check out the columns to make sure that they're all there
combined_2017.columns

Index(['state', '2017_act_participation', '2017_act_english', '2017_act_math',
       '2017_act_reading', '2017_act_science', '2017_act_composite',
       '2017_sat_participation', '2017_sat_evidence-based_reading_and_writing',
       '2017_sat_math', '2017_sat_total'],
      dtype='object')

In [44]:
# save cleaned, merged dataframe to csv
combined_2017.to_csv('../data/combined_2017.csv')